<a href="https://colab.research.google.com/github/keshav22bansal/BAKSA_IITK/blob/master/constraint_baseline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')  
PROJECT_PATH = "/content/gdrive/My Drive/Constraint/"

Mounted at /content/gdrive


In [ ]:
%mkdir -p "{PROJECT_PATH}"
%cd "{PROJECT_PATH}"
!git clone https://github.com/keshav22bansal/covid_fake_news_constraint

/content/gdrive/My Drive/Constraint
fatal: destination path 'covid_fake_news_constraint' already exists and is not an empty directory.


In [ ]:
import os
print(PROJECT_PATH)
data_path = os.path.join(PROJECT_PATH,"covid_fake_news_constraint/dataset")
train_name = "Constraint_English_Train.csv"
validation_name = "Constraint_English_Val.csv"
model_save_name = "constraint_baseline1.model"
bert_model = "bert-base-uncased"

/content/gdrive/My Drive/Constraint/


In [ ]:
import sys
sys.path.append(data_path)


In [ ]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

!pip install transformers

     |████████████████████████████████| 1.3MB 9.9MB/s 
     |████████████████████████████████| 2.9MB 50.7MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
     |████████████████████████████████| 1.1MB 46.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=c7d35b6124c19213a115a7870befd11210695c9b62d075dd9c9459393f2c2d3b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(bert_model)

In [ ]:
len(tokenizer.vocab)
tokens = tokenizer.tokenize('how ARE you ')

print(tokens)
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)


['how', 'are', 'you']
[2129, 2024, 2017]


In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)


# We can get the indexes of the special tokens by converting them using the vocabulary...

# In[7]:


init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# ...or by explicitly getting them from the tokenizer.

# In[8]:


init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)


# Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

# In[9]:


max_input_length = tokenizer.max_model_input_sizes[bert_model]

print(max_input_length)


# Previously we have used the `spaCy` tokenizer to tokenize our examples. However we now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

# In[10]:


def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    min_length = 5
    if len(tokens) < min_length:
      tokens = [init_token]*min_length
    return tokens


# Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.
# 
# We define the label field as before.

# In[87]:


from torchtext import data
data

[CLS] [SEP] [PAD] [UNK]
101 102 0 100
101 102 0 100
512


<module 'torchtext.data' from '/usr/local/lib/python3.6/dist-packages/torchtext/data/__init__.py'>

In [ ]:
UID = data.Field(sequential=False, use_vocab=False, pad_token=None)
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField()

# We load the data and create the validation splits as before.

# In[89]:


# from torchtext import datasets
# fields = {'uid':('uid',UID),'text': ('text', TEXT), 'label': ('label', LABEL) }
fields = [('id',UID),('tweet', TEXT),('label', LABEL)]
train_data, test_data = data.TabularDataset.splits(
                                        path = data_path,
                                        train = train_name,
                                        test = validation_name,
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)
# train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
# valid_data = train_data
train_data, valid_data = train_data.split(0.8,random_state = random.seed(SEED))


# In[90]:


# print(vars(train_data[1]))
# print(tokenizer.convert_ids_to_tokens(vars(test_data[331])))


In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")


# We can check an example and ensure that the text has already been numericalized.

# In[92]:


# print(vars(train_data.examples[0])['label'])
print(vars(train_data.examples[1]))
print(vars(train_data.examples[2]))



# We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

# In[93]:


tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[0])['tweet'])

print(tokens)


# Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

# In[94]:

UID.build_vocab(train_data)
LABEL.build_vocab(train_data)


# In[95]:


print(LABEL.vocab.stoi)
print(UID.vocab.stoi)

# As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

# In[96]:


BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Number of training examples: 5136
Number of validation examples: 1284
Number of testing examples: 2140
{'id': '3611', 'tweet': [1001, 14234, 6494, 3619, 24759, 4630, 2478, 1048, 24335, 8458, 10085, 17250, 2139, 10814, 3436, 15946, 1004, 23713, 1025, 3115, 6032, 10047, 23041, 2891, 6279, 20110, 3258, 2012, 1996, 4578, 1997, 1996, 1001, 2522, 17258, 16147, 6090, 3207, 7712, 1999, 1001, 16392, 16770, 1024, 1013, 1013, 1056, 1012, 2522, 1013, 6227, 16558, 2015, 3501, 20974, 2692, 2050, 1030, 17306, 13663, 2015, 18827, 1030, 7020, 14971, 27967, 1030, 2003, 7382, 5488, 2094, 5420, 1030, 19363, 3981, 4313, 20492, 2072, 1030, 14383, 11493, 8113, 2100, 2278, 1030, 24582, 4430, 2078, 20048, 11493, 4886, 1001, 5107, 7875, 20528, 6593, 16770, 1024, 1013, 1013, 1056, 1012, 2522, 1013, 1054, 2140, 4160, 16275, 5558, 2546, 3501, 2549], 'label': 'fake'}
{'id': '958', 'tweet': [3188, 2000, 1996, 3171, 2091, 22299, 3303, 2011, 2522, 17258, 1011, 2539, 1010, 1996, 2142, 2163, 4610, 2001, 2012, 2049, 4602

In [ ]:
device

device(type='cuda')

In [ ]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key=lambda x: len(x.tweet), 
    batch_size = BATCH_SIZE, 
    device = device)



# class_sample_count = [3,2,1] # dataset has 10 class-1 samples, 1 class-2 samples, etc.
# weights = 1 / torch.Tensor(class_sample_count).double()
# sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, BATCH_SIZE)
# train_iterator = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, sampler = sampler)
# valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size = BATCH_SIZE)
# valid_iterator = torch.utils.data.DataLoader(valid_data, batch_size = BATCH_SIZE)
# test_iterator = torch.utils.data.DataLoader(test_data, batch_size = BATCH_SIZE)

# print(type(train_iterator))

In [ ]:
# class_sample_count = [3,2,1] # dataset has 10 class-1 samples, 1 class-2 samples, etc.
# weights = 1 / torch.Tensor(class_sample_count).double()
# sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, BATCH_SIZE)
# train_iterator = torch.utils.data.DataLoader(train_iterator, batch_size = BATCH_SIZE, sampler = sampler)

In [ ]:

from transformers import BertTokenizer, BertModel
bert = BertModel.from_pretrained(bert_model)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        # embedding_dim = bert.config.to_dict()['dim']

        # self.conv_0 = nn.Conv2d(in_channels = 1, 
        #                         out_channels = n_filters, 
        #                         kernel_size = (filter_sizes, embedding_dim))
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        # self.conv = nn.Conv1d(74,74, 3)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
        #print(embedded.size())
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        #hidden = [n layers * n directions, batch size, emb dim]
        #print(hidden.size())
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #print(hidden.size())
        #hidden = [batch size, hid dim]
        # hidden = hidden.unsqueeze(1)
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output
class BERTCNNSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,n_filters,filter_sizes):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']-3
        self.conv_0 = nn.Conv2d(in_channels = 1, 
                                out_channels = 1, 
                                kernel_size = (2, 2))
        self.conv_1 = nn.Conv2d(in_channels = 1, 
                                out_channels = 1, 
                                kernel_size = (3, 3))

        self.pool = nn.MaxPool2d((2,1), stride=(2,1))
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)        

    def forward(self, text):
        
        #text = [batch size, sent len]
        # print(text.shape)
        with torch.no_grad():
            embedded = self.bert(text)[0]
        #print(embedded.size())
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        # print(embedded.size())
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        # print(pooled_0.size())
        # print(conved_0.shape)
        conved_1 = F.relu(self.conv_1(conved_0).squeeze(3))

        # print(self.conv_0(embedded).squeeze(3).size())
        # print(self.conv_0(embedded).size())

        # print(conved_0.size())
        # conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        # conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        
        pooled_1 = conved_1.squeeze(1)
        pooled_1 = self.dropout(pooled_1)
        # print(pooled_0.size())

        _, hidden = self.rnn(pooled_1)
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        output = self.out(hidden)
        
        
        return output

In [ ]:
HIDDEN_DIM = 100
OUTPUT_DIM = 2
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.5
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
# model = BERTGRUSentiment(bert,
#                          HIDDEN_DIM,
#                          OUTPUT_DIM,
#                          N_LAYERS,
#                          BIDIRECTIONAL,
#                          DROPOUT)

model = BERTCNNSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT,N_FILTERS,FILTER_SIZES)
# We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

# In[100]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


# In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

# In[101]:


for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False


# We can now see that our model has under 3M trainable parameters, making it almost comparable to the `FastText` model. However, the text still has to propagate through the transformer which causes training to take considerably longer.

# In[102]:


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 110,002,857 trainable parameters
The model has 520,617 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
# optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

# In[105]:


criterion = nn.CrossEntropyLoss()


# Place the model and criterion onto the GPU (if available)

# In[106]:


model = model.to(device)
criterion = criterion.to(device)


# Next, we'll define functions for: calculating accuracy, performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

# In[107]:




In [ ]:

def categorical_accuracy(preds, y):
    count0,count1,count2 = torch.zeros(1),torch.zeros(1),torch.zeros(1)
    total0,total1,total2 = torch.FloatTensor(1),torch.FloatTensor(1),torch.FloatTensor(1)
    # max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    f1 = f1_score(preds.argmax(dim = 1,keepdim = True).squeeze(1).cpu(),y.cpu())
    cnf = confusion_matrix(preds.argmax(dim = 1,keepdim = True).squeeze(1).cpu(),y.cpu())

    return f1,cnf
    # correct = max_preds.squeeze(1).eq(y)
    # predictions = max_preds.squeeze(1)F1 score
    # true_correct = [0,0,0]
    # for j,i in enumerate(y.cpu().numpy()):
    #   true_correct[y.cpu().numpy()[j]]+=1
    #   if i==0:
    #     count0+=correct[j]
    #     total0+=1
    #   elif i==1:
    #     count1+=correct[j]
    #     total1+=1
    #   elif i==2:
    #     count2+=correct[j]
    #   else:
    #     total2+=1
    # metric=torch.FloatTensor([count0/true_correct[0],count1/true_correct[1],count2/true_correct[2],f1_score(y.cpu().numpy(),predictions.cpu().numpy(),average='macro')])
    return correct.sum() / torch.FloatTensor([y.shape[0]]),metric,confusion_matrix(y.cpu().numpy(),max_preds.cpu().numpy())


# In[108]:


def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.tweet).squeeze(1)

        # print(predictions)


        loss = criterion(predictions, batch.label)

        # print('loss',loss)
        # (predictions.argmax(dim = 1) == batch.label).argmax(dim = 1)
        # acc = (predictions.argmax(dim = 1) == batch.label).argmax(dim = 1).sum().item()
        # acc = loss
        acc,cnf = categorical_accuracy(predictions, batch.label)
        # print(cnf)
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_all_acc = torch.FloatTensor([0,0,0,0])
    confusion_mat = torch.zeros((2,2))
    confusion_mat_temp = torch.zeros((2,2))

    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.tweet).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            # acc = all_acc = confusion_mat_temp = loss
            acc,confusion_mat_temp = categorical_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            # epoch_all_acc += all_acc.item()
            confusion_mat+=confusion_mat_temp
    return epoch_loss / len(iterator), epoch_acc / len(iterator),confusion_mat/len(iterator)


# In[110]:


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [ ]:
N_EPOCHS = 20

# best_valid_loss = float('inf')
best_f1 = -1
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc,conf = evaluate(model, valid_iterator, criterion)
    
    f1 = valid_acc

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # if valid_loss < best_valid_loss:
    #     best_valid_loss = valid_loss
    #     # torch.save(model.state_dict(), model_save_name)
    #     path = os.path.join(data_path,model_save_name)
    #     torch.save(model.state_dict(), path)
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), model_save_name)
        path = os.path.join(data_path,model_save_name)
        torch.save(model.state_dict(), path)
        
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    # print(tot)
    print(conf)


Epoch: 01 | Epoch Time: 0m 54s
	Train Loss: 0.254 | Train Acc: 88.84%
	 Val. Loss: 0.319 |  Val. Acc: 80.86%
tensor([[59.5455, 12.1818],
        [ 1.1818, 43.8182]], dtype=torch.float64)


In [ ]:
path = data_path+F"{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:

def test_evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    epoch_all_acc = torch.FloatTensor([[0,0],[0,0]])
    print(epoch_all_acc)
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.tweet).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc,all_acc = test_categorical_accuracy(predictions, batch.label)
            # print('**',all_acc)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_all_acc += all_acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator),epoch_all_acc/len(iterator)

from sklearn.metrics import confusion_matrix,f1_score
def test_categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # count0,count1 = torch.zeros(1),torch.zeros(1)
    # total0,total1 = torch.FloatTensor(1),torch.FloatTensor(1)
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    f1 = f1_score(preds.argmax(dim = 1,keepdim = True).squeeze(1).cpu(),y.cpu())
    cnf = confusion_matrix(preds.argmax(dim = 1,keepdim = True).squeeze(1).cpu(),y.cpu())

    return f1,cnf
    # correct = max_preds.squeeze(1).eq(y)
    # true_correct = [0,0,0]
    # for j,i in enumerate(y.cpu().numpy()):
    #   true_correct[y.cpu().numpy()[j]]+=1
    #   if i==0:
    #     count0+=correct[j]
    #     total0+=1
    #   elif i==1:
    #     count1+=correct[j]
    #     total1+=1
    #   elif i==2:
    #     count2+=correct[j]
    #   else:
    #     print(i,i==0,i==1,i==2)
    #     total2+=1
    # # print(count0,count1,count2,total0,total1,total2)
    # # print([count0/total0,count1/total1,count2/total2])
    # # print(torch.FloatTensor([count0/total0,count1/total1,count2/total2]))
    # # print(correct.sum() / torch.FloatTensor([y.shape[0]]))
    # # print(torch.FloatTensor([count0/total0,count1/total1,count2/total2]))
    # print(count0,count1,count2)
    # return correct.sum() / torch.FloatTensor([y.shape[0]]),torch.FloatTensor([count0/true_correct[0],count1/true_correct[1],count2/true_correct[2]])



In [ ]:
# model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc,test_all_acc = test_evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%',test_all_acc)


# ## Inference
# 
# We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

# In[ ]:







tensor([[0., 0.],
        [0., 0.]])
Test Loss: 0.189 | Test Acc: 91.18% tensor([[62.0588,  4.3529],
        [ 3.8235, 55.6471]], dtype=torch.float64)


In [ ]:
print(len(test_iterator))
24*128

24


3072

In [ ]:
print(len(test_data))

2998


In [ ]:
# {'1': 0, '2': 1, '0': 2})
m = inv_map = {v: k for k, v in LABEL.vocab.stoi.items()}
file = open('answer.txt','w')
file.write('Uid,Sentiment\n')
count = 0
for batch in test_iterator:
  predictions = model(batch.tweet).squeeze(1)
  max_preds = predictions.argmax(dim = 1, keepdim = True).detach().cpu().numpy()
  # print(batch.uid)
  # print(max_preds)
  for i,row in enumerate(batch.id.cpu().numpy()):
    count += 1
    print(count)
    if count != len(test_data):
      file.write('%s,%s\n'%(row,m[max_preds[i][0]]))
    else:
      file.write('%s,%s'%(row,m[max_preds[i][0]]))
    # if count != len(test_data):
    #   file.write('%s,%s,%s,%s\n'%(row," ".join(tokenizer.convert_)m[max_preds[i][0]]))
    # else:
    #   file.write('%s,%s'%(row,m[max_preds[i][0]]))
print(count)
file.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
len(l)
l


NameError: ignored

In [ ]:
f = open('')
for i in l:


In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    # ind = np.argmax(np.array(prediction))
    # if ind ==0:
    #   print('neutral')
    # elif ind == 1:
    #   print("positive")
    # else:
    #   print("negative")
    print(prediction)
predict_sentiment(model, tokenizer, "This film is terrible")


# In[ ]:


predict_sentiment(model, tokenizer, "This film is great")

In [ ]:
while True:
  sent = input('->')
  if sent != '$':
    predict_sentiment(model, tokenizer, sent)
  else:
    break
